# Purpose
- Check data for ROICat in LIMS

In [44]:
from pathlib import Path
import numpy as np
import pandas as pd
import h5py
import os
import glob
import json

from brain_observatory_qc.data_access import from_lims

from allensdk.brain_observatory.behavior.behavior_project_cache import VisualBehaviorOphysProjectCache as bpc

from pymongo import MongoClient
mongo = MongoClient("flaskapp.corp.alleninstitute.org", 27017)


In [2]:
# helper function
def get_session_info_per_group_of_mice(mouse_ids, gcamp_type):
    experiment_table = pd.DataFrame()
    problem_experiments = []
    for mouse_id in mouse_ids:
        mouse_data = from_lims.get_imaging_ids_for_mouse_id(mouse_id)
        if len(mouse_data) > 0:
            ophys_experiment_ids = mouse_data.ophys_experiment_id.values
            for ophys_experiment_id in ophys_experiment_ids: 
                try: 
                    expt_info = from_lims.get_general_info_for_ophys_experiment_id(ophys_experiment_id)
                    if len(expt_info) > 0:
                        expt_info = expt_info.iloc[[0]]
                        genotype = from_lims.get_genotype_for_ophys_experiment_id(ophys_experiment_id)
                        expt_info.insert(0, "mouse_id", mouse_id ) # expt_info only contains `donor_id` and `specimen_id`, need to add mouse_id here
                        expt_info.insert(1, "full_genotype", genotype) # base function doesnt get genotype, add it
                        experiment_table = pd.concat([experiment_table, expt_info])
                except: 
                    # print('problem for mouse_id: ', mouse_id, ', expt_id: ', ophys_experiment_id)
                    problem_experiments.append(ophys_experiment_id)
    experiment_table['gcamp'] = gcamp_type
    return experiment_table, problem_experiments


def get_mongo_client(username, password, host, port):
    client = MongoClient(f'mongodb://{username}:{password}@{host}:{port}')
    return client


def get_zdrift(opid):
    try:
        client = get_mongo_client('public', 'public_password', 'qc-sys-db', 27017)
    except:
        raise('Failed to connect to mongo')
    
    opid = int(opid)
    try:
        record = client.records.metrics.find_one({'data_id': opid})
    except:
        raise('No record found for opid: ', opid)
        
    try:
        zdrift = record['local_z_stack']['z_drift_corr_um_diff']
    except:
        zdrift = np.nan
    return zdrift


def get_monitor_sync(osid):
    try:
        client = get_mongo_client('public', 'public_password', 'qc-sys-db', 27017)
    except:
        raise('Failed to connect to mongo')
    
    osid = int(osid)
    try:
        record = client.records.metrics.find_one({'data_id': osid})
    except:
        raise('No record found for osid: ', osid)
        
    try:
        monitor_sync = record['rig_sync']['display_lag']
    except:
        monitor_sync = np.nan
    return monitor_sync

# One Slc32a1;Oi1 mouse from LIMS
- 721291

In [4]:
cache = bpc.from_lims()
table = cache.get_ophys_experiment_table(passed_only=False)

In [9]:
table.columns

Index(['equipment_name', 'donor_id', 'full_genotype', 'mouse_id',
       'reporter_line', 'driver_line', 'sex', 'age_in_days', 'foraging_id',
       'cre_line', 'indicator', 'session_number',
       'prior_exposures_to_session_type', 'prior_exposures_to_image_set',
       'prior_exposures_to_omissions', 'ophys_session_id',
       'behavior_session_id', 'ophys_container_id', 'project_code',
       'container_workflow_state', 'experiment_workflow_state', 'session_name',
       'isi_experiment_id', 'imaging_depth', 'targeted_structure',
       'published_at', 'date_of_acquisition', 'session_type',
       'experience_level', 'passive', 'image_set'],
      dtype='object')

In [10]:
mouse_table = table.query('mouse_id=="721291"')
# calculate order of depth in a given ophys_session_id
mouse_table['depth_order'] = mouse_table.groupby('ophys_session_id')['imaging_depth'].rank(method='dense', ascending=True)

C:\Users\jinho.kim\AppData\Local\Temp\ipykernel_19848\598891449.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mouse_table['depth_order'] = mouse_table.groupby('ophys_session_id')['imaging_depth'].rank(method='dense', ascending=True)


In [14]:
first_session_id = mouse_table.ophys_session_id.min()
first_session_table = mouse_table.query('ophys_session_id==@first_session_id')
first_session_table

,equipment_name,donor_id,full_genotype,mouse_id,reporter_line,driver_line,sex,age_in_days,foraging_id,cre_line,...,isi_experiment_id,imaging_depth,targeted_structure,published_at,date_of_acquisition,session_type,experience_level,passive,image_set,depth_order
ophys_experiment_id,,,,,,,,,,,,,,,,,,,,,
1345884085,MESO.2,1333456318,Slc32a1-IRES-Cre/wt;Oi1(TIT2L-jGCaMP8s-WPRE-IC...,721291,Oi1(TIT2L-jGCaMP8s-WPRE-ICL-IRES-tTA2),[Slc32a1-IRES-Cre],M,92.0,91619ed0-9246-493d-bc22-a23e8a5c242c,Slc32a1-IRES-Cre,...,1336899538,160,VISp,NaT,2024-04-16 15:21:39.646,TRAINING_0_gratings_autorewards_15min,None,False,i,4.0
1345884097,MESO.2,1333456318,Slc32a1-IRES-Cre/wt;Oi1(TIT2L-jGCaMP8s-WPRE-IC...,721291,Oi1(TIT2L-jGCaMP8s-WPRE-ICL-IRES-tTA2),[Slc32a1-IRES-Cre],M,92.0,91619ed0-9246-493d-bc22-a23e8a5c242c,Slc32a1-IRES-Cre,...,1336899538,314,VISp,NaT,2024-04-16 15:21:39.646,TRAINING_0_gratings_autorewards_15min,None,False,i,8.0
1345884091,MESO.2,1333456318,Slc32a1-IRES-Cre/wt;Oi1(TIT2L-jGCaMP8s-WPRE-IC...,721291,Oi1(TIT2L-jGCaMP8s-WPRE-ICL-IRES-tTA2),[Slc32a1-IRES-Cre],M,92.0,91619ed0-9246-493d-bc22-a23e8a5c242c,Slc32a1-IRES-Cre,...,1336899538,80,VISp,NaT,2024-04-16 15:21:39.646,TRAINING_0_gratings_autorewards_15min,None,False,i,2.0
1345884088,MESO.2,1333456318,Slc32a1-IRES-Cre/wt;Oi1(TIT2L-jGCaMP8s-WPRE-IC...,721291,Oi1(TIT2L-jGCaMP8s-WPRE-ICL-IRES-tTA2),[Slc32a1-IRES-Cre],M,92.0,91619ed0-9246-493d-bc22-a23e8a5c242c,Slc32a1-IRES-Cre,...,1336899538,120,VISp,NaT,2024-04-16 15:21:39.646,TRAINING_0_gratings_autorewards_15min,None,False,i,3.0
1345884093,MESO.2,1333456318,Slc32a1-IRES-Cre/wt;Oi1(TIT2L-jGCaMP8s-WPRE-IC...,721291,Oi1(TIT2L-jGCaMP8s-WPRE-ICL-IRES-tTA2),[Slc32a1-IRES-Cre],M,92.0,91619ed0-9246-493d-bc22-a23e8a5c242c,Slc32a1-IRES-Cre,...,1336899538,280,VISp,NaT,2024-04-16 15:21:39.646,TRAINING_0_gratings_autorewards_15min,None,False,i,7.0
1345884095,MESO.2,1333456318,Slc32a1-IRES-Cre/wt;Oi1(TIT2L-jGCaMP8s-WPRE-IC...,721291,Oi1(TIT2L-jGCaMP8s-WPRE-ICL-IRES-tTA2),[Slc32a1-IRES-Cre],M,92.0,91619ed0-9246-493d-bc22-a23e8a5c242c,Slc32a1-IRES-Cre,...,1336899538,40,VISp,NaT,2024-04-16 15:21:39.646,TRAINING_0_gratings_autorewards_15min,None,False,i,1.0
1345884087,MESO.2,1333456318,Slc32a1-IRES-Cre/wt;Oi1(TIT2L-jGCaMP8s-WPRE-IC...,721291,Oi1(TIT2L-jGCaMP8s-WPRE-ICL-IRES-tTA2),[Slc32a1-IRES-Cre],M,92.0,91619ed0-9246-493d-bc22-a23e8a5c242c,Slc32a1-IRES-Cre,...,1336899538,200,VISp,NaT,2024-04-16 15:21:39.646,TRAINING_0_gratings_autorewards_15min,None,False,i,5.0
1345884090,MESO.2,1333456318,Slc32a1-IRES-Cre/wt;Oi1(TIT2L-jGCaMP8s-WPRE-IC...,721291,Oi1(TIT2L-jGCaMP8s-WPRE-ICL-IRES-tTA2),[Slc32a1-IRES-Cre],M,92.0,91619ed0-9246-493d-bc22-a23e8a5c242c,Slc32a1-IRES-Cre,...,1336899538,240,VISp,NaT,2024-04-16 15:21:39.646,TRAINING_0_gratings_autorewards_15min,None,False,i,6.0


In [12]:
mouse_table.sort_values('date_of_acquisition')

,equipment_name,donor_id,full_genotype,mouse_id,reporter_line,driver_line,sex,age_in_days,foraging_id,cre_line,...,isi_experiment_id,imaging_depth,targeted_structure,published_at,date_of_acquisition,session_type,experience_level,passive,image_set,depth_order
ophys_experiment_id,,,,,,,,,,,,,,,,,,,,,
1345884090,MESO.2,1333456318,Slc32a1-IRES-Cre/wt;Oi1(TIT2L-jGCaMP8s-WPRE-IC...,721291,Oi1(TIT2L-jGCaMP8s-WPRE-ICL-IRES-tTA2),[Slc32a1-IRES-Cre],M,92.0,91619ed0-9246-493d-bc22-a23e8a5c242c,Slc32a1-IRES-Cre,...,1336899538,240,VISp,NaT,2024-04-16 15:21:39.646,TRAINING_0_gratings_autorewards_15min,None,False,i,6.0
1345884085,MESO.2,1333456318,Slc32a1-IRES-Cre/wt;Oi1(TIT2L-jGCaMP8s-WPRE-IC...,721291,Oi1(TIT2L-jGCaMP8s-WPRE-ICL-IRES-tTA2),[Slc32a1-IRES-Cre],M,92.0,91619ed0-9246-493d-bc22-a23e8a5c242c,Slc32a1-IRES-Cre,...,1336899538,160,VISp,NaT,2024-04-16 15:21:39.646,TRAINING_0_gratings_autorewards_15min,None,False,i,4.0
1345884097,MESO.2,1333456318,Slc32a1-IRES-Cre/wt;Oi1(TIT2L-jGCaMP8s-WPRE-IC...,721291,Oi1(TIT2L-jGCaMP8s-WPRE-ICL-IRES-tTA2),[Slc32a1-IRES-Cre],M,92.0,91619ed0-9246-493d-bc22-a23e8a5c242c,Slc32a1-IRES-Cre,...,1336899538,314,VISp,NaT,2024-04-16 15:21:39.646,TRAINING_0_gratings_autorewards_15min,None,False,i,8.0
1345884091,MESO.2,1333456318,Slc32a1-IRES-Cre/wt;Oi1(TIT2L-jGCaMP8s-WPRE-IC...,721291,Oi1(TIT2L-jGCaMP8s-WPRE-ICL-IRES-tTA2),[Slc32a1-IRES-Cre],M,92.0,91619ed0-9246-493d-bc22-a23e8a5c242c,Slc32a1-IRES-Cre,...,1336899538,80,VISp,NaT,2024-04-16 15:21:39.646,TRAINING_0_gratings_autorewards_15min,None,False,i,2.0
1345884088,MESO.2,1333456318,Slc32a1-IRES-Cre/wt;Oi1(TIT2L-jGCaMP8s-WPRE-IC...,721291,Oi1(TIT2L-jGCaMP8s-WPRE-ICL-IRES-tTA2),[Slc32a1-IRES-Cre],M,92.0,91619ed0-9246-493d-bc22-a23e8a5c242c,Slc32a1-IRES-Cre,...,1336899538,120,VISp,NaT,2024-04-16 15:21:39.646,TRAINING_0_gratings_autorewards_15min,None,False,i,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1369643473,MESO.2,1333456318,Slc32a1-IRES-Cre/wt;Oi1(TIT2L-jGCaMP8s-WPRE-IC...,721291,Oi1(TIT2L-jGCaMP8s-WPRE-ICL-IRES-tTA2),[Slc32a1-IRES-Cre],M,135.0,0f328c8a-7005-4c63-830c-e7b6b7eb0190,Slc32a1-IRES-Cre,...,1336899538,302,VISp,NaT,2024-05-29 15:46:35.000,STAGE_0,None,False,N/A,8.0
1369643464,MESO.2,1333456318,Slc32a1-IRES-Cre/wt;Oi1(TIT2L-jGCaMP8s-WPRE-IC...,721291,Oi1(TIT2L-jGCaMP8s-WPRE-ICL-IRES-tTA2),[Slc32a1-IRES-Cre],M,135.0,0f328c8a-7005-4c63-830c-e7b6b7eb0190,Slc32a1-IRES-Cre,...,1336899538,188,VISp,NaT,2024-05-29 15:46:35.000,STAGE_0,None,False,N/A,5.0
1369643468,MESO.2,1333456318,Slc32a1-IRES-Cre/wt;Oi1(TIT2L-jGCaMP8s-WPRE-IC...,721291,Oi1(TIT2L-jGCaMP8s-WPRE-ICL-IRES-tTA2),[Slc32a1-IRES-Cre],M,135.0,0f328c8a-7005-4c63-830c-e7b6b7eb0190,Slc32a1-IRES-Cre,...,1336899538,80,VISp,NaT,2024-05-29 15:46:35.000,STAGE_0,None,False,N/A,2.0


In [15]:
# Choose depth-order 3 (targeting 120 um)
plane_table = mouse_table.query('depth_order==3')
plane_table

,equipment_name,donor_id,full_genotype,mouse_id,reporter_line,driver_line,sex,age_in_days,foraging_id,cre_line,...,isi_experiment_id,imaging_depth,targeted_structure,published_at,date_of_acquisition,session_type,experience_level,passive,image_set,depth_order
ophys_experiment_id,,,,,,,,,,,,,,,,,,,,,
1367440728,MESO.2,1333456318,Slc32a1-IRES-Cre/wt;Oi1(TIT2L-jGCaMP8s-WPRE-IC...,721291,Oi1(TIT2L-jGCaMP8s-WPRE-ICL-IRES-tTA2),[Slc32a1-IRES-Cre],M,124.0,a039b9b6-de83-4d64-b0d0-e0e4186bc08c,Slc32a1-IRES-Cre,...,1336899538,114,VISp,NaT,2024-05-18 15:55:42.396,OPHYS_6_images_B,Novel >1,False,B,3.0
1345884088,MESO.2,1333456318,Slc32a1-IRES-Cre/wt;Oi1(TIT2L-jGCaMP8s-WPRE-IC...,721291,Oi1(TIT2L-jGCaMP8s-WPRE-ICL-IRES-tTA2),[Slc32a1-IRES-Cre],M,92.0,91619ed0-9246-493d-bc22-a23e8a5c242c,Slc32a1-IRES-Cre,...,1336899538,120,VISp,NaT,2024-04-16 15:21:39.646,TRAINING_0_gratings_autorewards_15min,None,False,i,3.0
1346117513,MESO.2,1333456318,Slc32a1-IRES-Cre/wt;Oi1(TIT2L-jGCaMP8s-WPRE-IC...,721291,Oi1(TIT2L-jGCaMP8s-WPRE-ICL-IRES-tTA2),[Slc32a1-IRES-Cre],M,93.0,9d7d4a89-8869-4e32-94a0-ca2a58fd9537,Slc32a1-IRES-Cre,...,1336899538,120,VISp,NaT,2024-04-17 18:02:16.579,TRAINING_1_gratings,None,False,i,3.0
1347164369,MESO.2,1333456318,Slc32a1-IRES-Cre/wt;Oi1(TIT2L-jGCaMP8s-WPRE-IC...,721291,Oi1(TIT2L-jGCaMP8s-WPRE-ICL-IRES-tTA2),[Slc32a1-IRES-Cre],M,98.0,2a464073-2b49-4d27-a8a1-05f9d01f1b5a,Slc32a1-IRES-Cre,...,1336899538,120,VISp,NaT,2024-04-22 15:38:58.372,TRAINING_1_gratings,None,False,i,3.0
1348170438,MESO.2,1333456318,Slc32a1-IRES-Cre/wt;Oi1(TIT2L-jGCaMP8s-WPRE-IC...,721291,Oi1(TIT2L-jGCaMP8s-WPRE-ICL-IRES-tTA2),[Slc32a1-IRES-Cre],M,100.0,7569ba48-7505-4dd9-93e1-73bae2da8543,Slc32a1-IRES-Cre,...,1336899538,122,VISp,NaT,2024-04-24 15:13:28.966,TRAINING_2_gratings_flashed,None,False,i,3.0
1348159915,MESO.2,1333456318,Slc32a1-IRES-Cre/wt;Oi1(TIT2L-jGCaMP8s-WPRE-IC...,721291,Oi1(TIT2L-jGCaMP8s-WPRE-ICL-IRES-tTA2),[Slc32a1-IRES-Cre],M,99.0,f65a9dad-c5b8-4a8f-b7ba-3e842538a69a,Slc32a1-IRES-Cre,...,1336899538,118,VISp,NaT,2024-04-23 15:20:25.309,TRAINING_2_gratings_flashed,None,False,i,3.0
1348170463,MESO.2,1333456318,Slc32a1-IRES-Cre/wt;Oi1(TIT2L-jGCaMP8s-WPRE-IC...,721291,Oi1(TIT2L-jGCaMP8s-WPRE-ICL-IRES-tTA2),[Slc32a1-IRES-Cre],M,102.0,a9d9d2fe-bf4d-4269-8ff7-47c956255314,Slc32a1-IRES-Cre,...,1336899538,122,VISp,NaT,2024-04-26 15:05:27.480,TRAINING_3_images_A_10uL_reward,None,False,e,3.0
1363153133,MESO.2,1333456318,Slc32a1-IRES-Cre/wt;Oi1(TIT2L-jGCaMP8s-WPRE-IC...,721291,Oi1(TIT2L-jGCaMP8s-WPRE-ICL-IRES-tTA2),[Slc32a1-IRES-Cre],M,105.0,6fd4c52d-348d-49a3-a268-720fe8532aa7,Slc32a1-IRES-Cre,...,1336899538,120,VISp,NaT,2024-04-29 15:28:28.051,TRAINING_3_images_A_10uL_reward,None,False,e,3.0
1363363116,MESO.2,1333456318,Slc32a1-IRES-Cre/wt;Oi1(TIT2L-jGCaMP8s-WPRE-IC...,721291,Oi1(TIT2L-jGCaMP8s-WPRE-ICL-IRES-tTA2),[Slc32a1-IRES-Cre],M,106.0,68c33d44-6b60-41a1-a813-a81fceb2426d,Slc32a1-IRES-Cre,...,1336899538,120,VISp,NaT,2024-04-30 15:46:46.008,TRAINING_3_images_A_10uL_reward,None,False,e,3.0


In [17]:
# add experiment_directory
oeid = 1369433939
from_lims.get_general_info_for_ophys_experiment_id(oeid).columns

Index(['ophys_experiment_id', 'ophys_session_id', 'behavior_session_id',
       'foraging_id', 'ophys_container_id', 'supercontainer_id',
       'experiment_workflow_state', 'session_workflow_state',
       'container_workflow_state', 'specimen_id', 'donor_id', 'specimen_name',
       'date_of_acquisition', 'session_type', 'targeted_structure', 'depth',
       'equipment_name', 'project', 'experiment_storage_directory',
       'behavior_storage_directory', 'session_storage_directory',
       'container_storage_directory', 'supercontainer_storage_directory',
       'specimen_storage_directory'],
      dtype='object')

In [27]:
for oeid, row in plane_table.iterrows():
    oeid_dir = from_lims.get_general_info_for_ophys_experiment_id(oeid)['experiment_storage_directory'].values[0]
    plane_table.loc[oeid, 'experiment_storage_directory'] = oeid_dir

\\allen\programs\mindscope\production\learning\prod0\specimen_1333456325\ophys_session_1367326383\ophys_experiment_1367440728
\\allen\programs\mindscope\production\learning\prod0\specimen_1333456325\ophys_session_1345647212\ophys_experiment_1345884088
\\allen\programs\mindscope\production\learning\prod0\specimen_1333456325\ophys_session_1345978020\ophys_experiment_1346117513
\\allen\programs\mindscope\production\learning\prod0\specimen_1333456325\ophys_session_1347023256\ophys_experiment_1347164369
\\allen\programs\mindscope\production\learning\prod0\specimen_1333456325\ophys_session_1347439922\ophys_experiment_1348170438
\\allen\programs\mindscope\production\learning\prod0\specimen_1333456325\ophys_session_1347238498\ophys_experiment_1348159915
\\allen\programs\mindscope\production\learning\prod0\specimen_1333456325\ophys_session_1347943754\ophys_experiment_1348170463
\\allen\programs\mindscope\production\learning\prod0\specimen_1333456325\ophys_session_1348625182\ophys_experiment_136

In [32]:
import h5py 
with h5py.File(oeid_dir / 'roi_traces.h5') as h:
    print(h.keys())

<KeysViewHDF5 ['data', 'roi_names']>


In [36]:
# add filenames to average projection, maximum projection, and roi data
def get_roi_json_fn(oeid_dir):
    roi_json_fn = glob.glob(str(oeid_dir / '*_OPHYS_EXTRACT_TRACES_*_output.json'))[0]
    return roi_json_fn

def get_average_projection_fn(oeid_dir):
    average_projection_fn = glob.glob(str(oeid_dir / 'processed' / '*_suite2p_average_projection.png'))[0]
    return average_projection_fn

def get_max_projection_fn(oeid_dir):
    max_projection_fn = glob.glob(str(oeid_dir / 'processed' / '*_suite2p_maximum_projection.png'))[0]
    return max_projection_fn


for oeid, row in plane_table.iterrows():
    oeid_dir = from_lims.get_general_info_for_ophys_experiment_id(oeid)['experiment_storage_directory'].values[0]
    plane_table.loc[oeid, 'experiment_storage_directory'] = oeid_dir
    plane_table.loc[oeid, 'roi_json_fn'] = get_roi_json_fn(oeid_dir)
    plane_table.loc[oeid, 'average_projection_fn'] = get_average_projection_fn(oeid_dir)
    plane_table.loc[oeid, 'max_projection_fn'] = get_max_projection_fn(oeid_dir)

C:\Users\jinho.kim\AppData\Local\Temp\ipykernel_19848\2372725355.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plane_table.loc[oeid, 'roi_json_fn'] = get_roi_json_fn(oeid_dir)
C:\Users\jinho.kim\AppData\Local\Temp\ipykernel_19848\2372725355.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plane_table.loc[oeid, 'average_projection_fn'] = get_average_projection_fn(oeid_dir)
C:\Users\jinho.kim\AppData\Local\Temp\ipykernel_19848\2372725355.py:20: SettingWithCopyWarning: 
A value is trying to be set o

In [43]:
# Copy files
import shutil
target_dir_base = Path(r'\\allen\programs\mindscope\workgroups\learning\pilots\roicat')
target_dir = target_dir_base / '721291'
target_dir.mkdir(exist_ok=True)

for oeid, row in mouse_table.iterrows():
    oeid_dir = from_lims.get_general_info_for_ophys_experiment_id(oeid)['experiment_storage_directory'].values[0]
    roi_json_fn = get_roi_json_fn(oeid_dir)
    average_projection_fn = get_average_projection_fn(oeid_dir)
    max_projection_fn = get_max_projection_fn(oeid_dir)
    depth_order = int(row['depth_order'])
    target_depth_dir = target_dir / f'plane_{depth_order}'
    target_depth_dir.mkdir(exist_ok=True)
    
    shutil.copy(roi_json_fn, target_depth_dir / f'{oeid}_roi.json')
    shutil.copy(average_projection_fn, target_depth_dir / average_projection_fn.split('\\')[-1])
    shutil.copy(max_projection_fn, target_depth_dir / max_projection_fn.split('\\')[-1])


In [57]:
# Converting json to csv (only roi mask data, including validity)
for oeid, row in mouse_table.iterrows():
    depth_order = int(row['depth_order'])
    target_depth_dir = target_dir / f'plane_{depth_order}'
    roi_json_fn = target_depth_dir / f'{oeid}_roi.json'
    with open(roi_json_fn, 'r') as f:
        roi_data = json.load(f)
    roi_df = pd.DataFrame(roi_data['input_parameters']['rois']).set_index('id', drop=True).sort_index()[['x', 'y', 'width', 'height', 'mask', 'valid']]
    roi_df.to_csv(target_depth_dir / f'{oeid}_roi.csv')

# Make table for uploading and processing in CodeOcean


In [75]:
session_uploading_table = mouse_table[['ophys_session_id', 'date_of_acquisition', 'mouse_id', 'session_type']].drop_duplicates()
oeids_list = []
for i, row in session_uploading_table.iterrows():
    session_id = row['ophys_session_id']
    oeids = from_lims.get_ophys_experiment_ids_for_ophys_session_id(session_id).ophys_experiment_id.values  
    oeids_list.append(oeids)
    session_dir = from_lims.get_general_info_for_ophys_session_id(session_id).session_storage_directory.unique()[0]
    session_uploading_table.loc[i, 'session_storage_directory'] = session_dir
session_uploading_table['ophys_experiment_ids'] = oeids_list
session_uploading_table.reset_index(drop=True, inplace=True)



In [77]:
# Save to a csv file
session_uploading_table.date_of_acquisition = session_uploading_table.date_of_acquisition.dt.strftime('%Y-%m-%d_%H-%M-%S')
session_uploading_table.to_csv(target_dir / 'session_uploading_table.csv')

In [59]:
session_id = row['ophys_session_id']
from_lims.get_general_info_for_ophys_session_id(session_id).session_storage_directory.unique()

array([WindowsPath('//allen/programs/mindscope/production/learning/prod0/specimen_1333456325/ophys_session_1369312751')],
      dtype=object)

# How about pan-neuronal GCaMP8s data?
- Is there any data with at least 4 sessions?
    - Yes, 726087 (AAV local injection) and 726433 (SNAP25-Cre;Oi4)